In [8]:
# Load the saved data
using JLD
outp = load("BK_manual_model_10_to_14_reduct.jld", "outp");


In [9]:
# Load the BK functions and the x_grid(Ca, V)
include("../BK_functions/bk_setup_script.jl"); 
Ca_grid = reshape(hcat(x_grid...)[1,:], (8,13));
V_grid = reshape(hcat(x_grid...)[2,:], (8,13));

In [16]:
#=
using NBInclude
nbinclude("MBAM_plots.ipynb")

f_model(phi; model_id=0) = reshape(BK_simulator(phi0, hcat(x_grid...),model_id=model_id), (8,13));

p_orig = MBAM_plots.plot(V', f_model(phi0)', 
    marker_color=Any[1:13, "rgb(16,32,77)"],
    mode=["markers", "markers+lines","lines"]
    )

p_iter5 = MBAM_plots.plot(V', reshape(BK_simulator(exp(outp[1][5,1]), hcat(x_grid...),model_id=14), (8,13))');

a = PlotlyJS.Plot(p_orig, MBAM_plots.layouts())
b = PlotlyJS.Plot(p_iter5, MBAM_plots.layouts())



PlotlyJS.plot([a, b])
=#

In [108]:
nbinclude("MBAM_plots.ipynb")
import MBAM_plots

function create_boundary_iter_plot(outp; 
    param_names::AbstractArray=[], model_id=collect(10:14), Ca=Ca, V=V, x_grid=x_grid,
    log_plots=false)
    # From the output of the MBAM code, creates pretty plots automatically
    
    if isempty(param_names)
        param_names = Array(Array{AbstractString},size(outp[1],1));
        param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]
        param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]
        param_names[3] = ["J0", "zJ", "Kd", "L0^1/4 * C"," L0^1/4 * D", "E / L0^1/4"];
        param_names[4] = ["J0", "zJ", "Kd", "C * E"," L0^1/4 * D"];
        param_names[5] = ["J0", "zJ", "C * E/Kd"," L0^1/4 * D"];
    end
    
    
    
    color_list_Ca = ["rgb(0, $g, 0)" for g in 100:20:(100+7*20)]
    color_list_phi = ["rgb(200, 40, 40)", "rgb(200, 80, 80)", "rgb(40, 40, 200)", 
                      "rgb(80, 80, 200)", "rgb(40, 200, 40)", "rgb(120, 200, 120)",
                      "rgb(120, 120, 200)", "rgb(120, 200, 200)"]
    
    @show color_list_phi
    
    name_list_Ca = ["[Ca]=$(round(Ca[i1]*1e6)) &mu;mol" for i1 in 1:size(Ca,1)]
    
    model_outp_plots = Array(PlotlyJS.Plot{PlotlyJS.GenericTrace{Dict{Symbol,Any}}}, size(outp[1],1));
    boundary_iter_plots = Array(PlotlyJS.Plot{PlotlyJS.GenericTrace{Dict{Symbol,Any}}}, size(outp[1],1));
    
    if !log_plots
        for i1 = 1:size(outp[1],1)
            model_outp_plots[i1] = 
                PlotlyJS.Plot(
                    MBAM_plots.plot(V', reshape(BK_simulator(exp(outp[1][i1,1]), hcat(x_grid...),
                        model_id=model_id[i1]), size(V))',
            marker_color=color_list_Ca, showlegend=(i1==1), legendgroup="model", name=name_list_Ca
                    ),
                    MBAM_plots.layouts()
                )

            t_out = outp[5][i1];
            y_out = hcat(outp[6][i1]...)



            boundary_iter_plots[i1] = PlotlyJS.Plot(
                MBAM_plots.plot(t_out, y_out[1:div(size(y_out,1),2),:]',
            showlegend=true, legendgroup="phi_iter_$(i1)", name=param_names[i1],
            mode="markers", marker_color=color_list_phi, marker_size=3
            ),
                MBAM_plots.layouts("BK_MBAM_search")
            )
        end
    else
        for i1 = 1:size(outp[1],1)
            model_outp_plots[i1] = 
                PlotlyJS.Plot(
            MBAM_plots.plot(V', log10(reshape(BK_simulator(exp(outp[1][i1,1]), hcat(x_grid...),
                        model_id=model_id[i1]), size(V))'),
            marker_color=color_list_Ca, showlegend=(i1==1), legendgroup="model", name=name_list_Ca
                    ),
                    MBAM_plots.layouts()
                )
        end
    end
                
    
    return model_outp_plots, boundary_iter_plots
end


create_boundary_iter_plot (generic function with 1 method)

In [64]:
model_outp_plots, boundary_iter_plots = create_boundary_iter_plot(outp,
Ca=Ca_grid, V=V_grid, x_grid=x_grid);

color_list_phi = ASCIIString["rgb(200, 40, 40)","rgb(200, 80, 80)","rgb(40, 40, 200)","rgb(80, 80, 200)","rgb(40, 200, 40)","rgb(120, 200, 120)","rgb(120, 120, 200)","rgb(120, 200, 200)"]


In [66]:
# Now actually plot it
plt = PlotlyJS.plot([model_outp_plots[1] boundary_iter_plots[1]
    model_outp_plots[2] boundary_iter_plots[2]
    model_outp_plots[3] boundary_iter_plots[3]
    model_outp_plots[4] boundary_iter_plots[4]
    model_outp_plots[5] boundary_iter_plots[5]])

relayout!(plt, height=1600)

plt

In [67]:
PlotlyJS.savefig(plt, "BK_reduce.eps")

LoadError: LoadError: MethodError: `isactive` has no method matching isactive(::PlotlyJS.JupyterDisplay)
while loading In[67], in expression starting on line 1

# LOW CALCIUM CONDITION

In [68]:
outp_lowCa = load("partial_models_lowCa_iter5.jld", "outp_lowCa")

(
5x2 Array{Any,2}:
 [-13.027053197600004,-0.8675005677047232,-2.276917346245468,-0.544727175441672,-10.151948911834628,1.8180767775454285,3.414442608412176,0.6931471805599453]  …  [-13.144455650088121,-0.8897563976215435,-2.2860317730318758,-0.5507743768445709,-4.867383518040076,2.065247098436586,3.4651170787198526,5.6684298749374245]
 [-13.144455650088121,-0.8897563976215435,-2.2860317730318758,-0.5507743768445709,10.5358133929775,2.065247098436586,3.4651170787198526]                         [-41.206783709883965,-1.087720797817987,-2.279956714501311,-0.6143179644872516,9.015192077300961,3.488571820931338,10.654233563053225]                      
 [0.35253763558223383,-1.087720797817987,-2.279956714501311,-0.6143179644872516,9.015192077300961,3.488571820931338]                                             [0.35456180667583753,-1.0905109585886519,-2.279539271797785,-0.615419804299885,7.627286289057182,4.863156305634106]                                         
 [0.35456180667583753,-1.0

In [72]:
# Load the BK functions and the x_grid(Ca, V)
include("../BK_functions/bk_setup_script_lowCa.jl"); 
Ca_grid = reshape(hcat(x_grid...)[1,:], (2,13));
V_grid = reshape(hcat(x_grid...)[2,:], (2,13));

In [81]:
param_names = Array(Array{AbstractString},size(outp[1],1));
param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[2] = ["L0", "zL", "J0", "zJ", "E/Kd", "C", "D"]
param_names[3] = ["L0^(1/4)*D", "zL", "J0", "zJ", "E/Kd", "C"]
param_names[4] = ["L0^(1/4)*D", "zL", "J0", "zJ", "C*E/Kd"]
param_names[5] = ["L0^(1/4)*D", "J0", "zJ", "C*E/Kd"]

4-element Array{ASCIIString,1}:
 "L0^(1/4)*D"
 "J0"        
 "zJ"        
 "C*E/Kd"    

In [82]:
model_outp_plots, boundary_iter_plots = create_boundary_iter_plot(outp_lowCa; 
param_names=param_names, model_id=collect(30:34), Ca=Ca_grid, V=V_grid, x_grid=x_grid);

color_list_phi = ASCIIString["rgb(200, 40, 40)","rgb(200, 80, 80)","rgb(40, 40, 200)","rgb(80, 80, 200)","rgb(40, 200, 40)","rgb(120, 200, 120)","rgb(120, 120, 200)","rgb(120, 200, 200)"]


In [83]:
# Now actually plot it
plt = PlotlyJS.plot([model_outp_plots[1] boundary_iter_plots[1]
    model_outp_plots[2] boundary_iter_plots[2]
    model_outp_plots[3] boundary_iter_plots[3]
    model_outp_plots[4] boundary_iter_plots[4]
    model_outp_plots[5] boundary_iter_plots[5]])

relayout!(plt, height=1600)

plt

In [84]:
PlotlyJS.savefig(plt, "BK_lowCa_reduce.eps")

LoadError: LoadError: MethodError: `isactive` has no method matching isactive(::PlotlyJS.JupyterDisplay)
while loading In[84], in expression starting on line 1



# Very low voltage tail behaviour

Plotting log P_open probabilities for very low voltages

In [ ]:
println("Starting low voltage...")

In [109]:
# Setting up the longer voltage range
# Create grid to evaluate over
using Iterators
Ca=([0.0, 0.7, 4.0, 12.0, 22.0, 55.0, 70.0, 95.0]*1e-6)
V=collect(-175:25:200)*1e-3
Ca = float(Ca)
V = float(V)

x_grid = Array(Any, (length(Ca)*length(V),))
iterind = 0;
for j1 in product(Ca, V)
    iterind += 1;
    x_grid[iterind] = collect(j1)
end
size(x_grid)


Ca_grid = reshape(hcat(x_grid...)[1,:], length(Ca), length(V));
V_grid = reshape(hcat(x_grid...)[2,:], length(Ca), length(V));

In [110]:
# Get the extended plots

model_outp_plots, = create_boundary_iter_plot(outp; param_names=param_names, 
model_id=collect(10:14), Ca=Ca_grid, V=V_grid, x_grid=x_grid, log_plots=true);

color_list_phi = ASCIIString["rgb(200, 40, 40)","rgb(200, 80, 80)","rgb(40, 40, 200)","rgb(80, 80, 200)","rgb(40, 200, 40)","rgb(120, 200, 120)","rgb(120, 120, 200)","rgb(120, 200, 200)"]


In [111]:
# Now actually plot it
plt = PlotlyJS.plot([model_outp_plots[1]
    model_outp_plots[2]
    model_outp_plots[3]
    model_outp_plots[4]
    model_outp_plots[5]])

relayout!(plt, height=1600)

plt

In [ ]:
PlotlyJS.savefig(plt, "BK_reduce_tail.eps")



## Low Ca condition

In [106]:
model_outp_plots, = create_boundary_iter_plot(outp_lowCa; param_names=param_names, 
model_id=collect(30:34), Ca=Ca_grid, V=V_grid, x_grid=x_grid, log_plots=true);

color_list_phi = ASCIIString["rgb(200, 40, 40)","rgb(200, 80, 80)","rgb(40, 40, 200)","rgb(80, 80, 200)","rgb(40, 200, 40)","rgb(120, 200, 120)","rgb(120, 120, 200)","rgb(120, 200, 200)"]


In [107]:
# Now actually plot it
plt = PlotlyJS.plot([model_outp_plots[1]
    model_outp_plots[2]
    model_outp_plots[3]
    model_outp_plots[4]
    model_outp_plots[5]])

relayout!(plt, height=1600)

plt

In [ ]:
PlotlyJS.savefig(plt, "BK_lowCa_reduce_tail.eps")

In [123]:
exp(outp_lowCa[1][1,2])

8-element Array{Float64,1}:
   1.9563e-6 
   0.410756  
   0.101669  
   0.576503  
   0.00769347
   7.88725   
  31.9802    
 289.58      